In [1]:
from datasets import load_dataset

In [2]:
go_emotions = load_dataset("go_emotions")
data = go_emotions.data

No config specified, defaulting to: go_emotions/simplified
Reusing dataset go_emotions (C:\Users\rames\.cache\huggingface\datasets\go_emotions\simplified\0.0.0\2637cfdd4e64d30249c3ed2150fa2b9d279766bfcd6a809b9f085c61a90d776d)


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
train=data['train'].to_pandas()
validation=data['validation'].to_pandas()
test=data['test'].to_pandas()

In [4]:
import tqdm

In [5]:
import pandas as pd
label = []
for i in range(len(train)):
    label_indices = train.iloc[i]["labels"]
    label.append(label_indices[0]);

In [6]:
train = pd.concat([train,pd.DataFrame(label)],axis=1)

In [7]:
train.head()

,text,labels,id,0
0,Is this in New Orleans?? I really feel like th...,[27],edgurhb,27
1,"You know the answer man, you are programmed to...","[4, 27]",ee84bjg,4
2,I've never been this sad in my life!,[25],edcu99z,25
3,The economy is heavily controlled and subsidiz...,"[4, 27]",edc32e2,4
4,He could have easily taken a real camera from ...,[20],eepig6r,20


In [8]:
train.drop(['labels','id'],axis=1)

,text,0
0,Is this in New Orleans?? I really feel like th...,27
1,"You know the answer man, you are programmed to...",4
2,I've never been this sad in my life!,25
3,The economy is heavily controlled and subsidiz...,4
4,He could have easily taken a real camera from ...,20
...,...,...
5421,It's pretty dangerous when the state decides w...,14
5422,I filed for divorce this morning. Hoping he mo...,20
5423,"The last time it happened I just said, ""No"" an...",10
5424,I can’t stand this arrogant prick he’s no bett...,3


In [103]:
train.isnull().sum()

text      0
labels    0
id        0
0         0
dtype: int64

In [128]:
import pandas as pd
import numpy as np
import contractions
import string
import nltk
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer

In [127]:
train['processing'] = train['text'].apply(lambda x: [contractions.fix(word) for word in x.split()])
train['processing'] = train['processing'].apply(lambda x: [word.lower() for word in x])
train['processing'] = [' '.join(map(str,l)) for l in train['processing']]
train['processing'] = [re.sub(r'[^\w\s]', '', l) for l in train['processing']]
train['processing'] = train['processing'].apply(word_tokenize)

In [134]:
stop_words = set(stopwords.words('english'))
train['processing'] = train['processing'].apply(lambda x: [word for word in x if word not in stop_words])

In [139]:
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

train['processing'] = train['processing'].apply(nltk.tag.pos_tag)
train['processing'] = train['processing'].apply(lambda x: [(word,get_wordnet_pos(word)) for (word,pos_tag) in x])

In [141]:
wnl = WordNetLemmatizer()
train['processing'] = train['processing'].apply(lambda x:[wnl.lemmatize(word,tag) for (word,tag) in x])
train['processing'] =[' '.join(l) for l in train['processing']]

In [142]:
train.head()

,text,labels,id,0,processing
0,My favourite food is anything I didn't have to...,[27],eebbqej,27,favourite food anything cook
1,"Now if he does off himself, everyone will thin...",[27],ed00q6i,27,everyone think he laugh screwing people instea...
2,WHY THE FUCK IS BAYLESS ISOING,[2],eezlygj,2,fuck bayless isoing
3,To make her feel threatened,[14],ed7ypvh,14,make feel threatened
4,Dirty Southern Wankers,[3],ed0bdzj,3,dirty southern wanker


In [145]:
train = train[['text',0,'processing']]
train.to_csv('/Users/rames/Documents/IITB/Others/SOC 2022/reddit_processed_data.csv', encoding='utf-8', index=False)

In [ ]:
# I should remove [name] in the start only and then numbers in between the comments are still there